# 0.0 Imports

In [1]:
import os
import requests
import pandas         as pd
import numpy          as np
import seaborn        as sns
import plotly.express as px
import ipywidgets     as widgets
import zipfile     

from io                    import BytesIO
from ipywidgets            import fixed
from matplotlib            import gridspec
from matplotlib            import pyplot as plt
from geopy.geocoders       import Nominatim
from IPython.core.display  import HTML
from IPython.display       import Image

## 0.1 Helper functions

In [2]:
# Estatística Descritiva
def num_metricas(num_attributes):
    
  #Central tendencian - mean, median
    ct1 = pd.DataFrame( num_attributes.apply(np.mean)).T
    ct2 = pd.DataFrame(num_attributes.apply(np.median)).T

  #Dispersion - std, min, max, range, skew, kurtosis

    d1 = pd.DataFrame(num_attributes.apply(np.std)).T
    d2 = pd.DataFrame(num_attributes.apply(min)).T
    d3 = pd.DataFrame(num_attributes.apply(max)).T
    d4 = pd.DataFrame(num_attributes.apply(lambda x: x.max() - x.min() )).T
    d5 = pd.DataFrame(num_attributes.apply(lambda x: x.skew() )).T
    d6 = pd.DataFrame(num_attributes.apply(lambda x: x.kurtosis() )).T

  #Concatenate
    metrics = pd.concat([d2, d3, d4, ct1, ct2, d1, d5, d6]).T.reset_index()
    metrics.columns = ['attributes','min', 'max', 'range', 'mean', 'median', 'std', 'skew','kurtosis']
  
    return metrics

def jupyter_settings():
    
    %matplotlib inline 
    #%pylab inline
    
    plt.style.use('bmh')
    plt.rcParams['figure.figsize']=[20,10]
    plt.rcParams['font.size']=10
    
    display( HTML('<style>.container {width:100% !important; }</style>'))
    pd.options.display.max_columns = None
    pd.options.display.max_rows = None
    pd.set_option('display.expand_frame_repr',False )
    pd.set_option('display.float_format',lambda x: '%.2f' % x)
    

    sns.set()
    


jupyter_settings()

## 0.2 Loading Data

In [3]:
df_raw = pd.read_csv('data/kc_house_data.csv',low_memory=False)
df = df_raw.copy()
df.sample(5)

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
4910,339600460,20141017T000000,419500.00,3,2.50,1360,3188,2.00,0,0,3,7,1360,0,1986,0,98052,47.68,-122.10,1090,3188
5454,9413600420,20140612T000000,890000.00,3,2.25,2060,8640,1.00,0,0,4,8,2060,0,1966,0,98033,47.65,-122.19,2030,9000
16801,9197100101,20150504T000000,225000.00,2,1.00,1010,5408,1.00,0,0,4,6,1010,0,1926,0,98032,47.38,-122.24,980,7800
20880,3362400092,20150312T000000,565000.00,3,2.25,1540,1005,3.00,0,0,3,8,1540,0,2008,0,98103,47.68,-122.35,1510,1501
7752,4046700210,20140629T000000,345000.00,3,2.00,1610,15005,1.00,0,0,4,7,1610,0,1986,0,98014,47.69,-121.91,1610,15479


# 1.0 Descrição dos Dados

## 1.1. Dimensão dos Dados

In [4]:
print('Quantidade de linhas: {}'.format(df.shape[0]))
print('Quantidade de colunas: {}'.format(df.shape[1]))

Quantidade de linhas: 21613
Quantidade de colunas: 21


## 1.2. Tipos dos Dados

In [5]:
df.dtypes

id                 int64
date              object
price            float64
bedrooms           int64
bathrooms        float64
sqft_living        int64
sqft_lot           int64
floors           float64
waterfront         int64
view               int64
condition          int64
grade              int64
sqft_above         int64
sqft_basement      int64
yr_built           int64
yr_renovated       int64
zipcode            int64
lat              float64
long             float64
sqft_living15      int64
sqft_lot15         int64
dtype: object

Podemos observar que a variável 'date' apresenta o tipo object como é uma data vamos realizar a mudança do tipo de variável.

In [6]:
df['date'] = pd.to_datetime(df['date'],errors='coerce')

## 1.3 Check NA

In [7]:
df.isna().sum()

id               0
date             0
price            0
bedrooms         0
bathrooms        0
sqft_living      0
sqft_lot         0
floors           0
waterfront       0
view             0
condition        0
grade            0
sqft_above       0
sqft_basement    0
yr_built         0
yr_renovated     0
zipcode          0
lat              0
long             0
sqft_living15    0
sqft_lot15       0
dtype: int64

**Obs:** O dataset não apresenta valores nulos.

## 1.4 Estatística Descritiva

In [8]:
#Separação das variáveis numéricas e categóricas
num_atributos = df.select_dtypes(include = ['int64', 'float64'])
cat_atributos = df.select_dtypes(include = ['object'])

### 1.4.1 Variáveis Numéricas

In [9]:
 num_metricas(num_atributos)

,attributes,min,max,range,mean,median,std,skew,kurtosis
0,id,1000102.00,9900000190.00,9899000088.00,4580301520.86,3904930410.00,2876499023.43,0.24,-1.26
1,price,75000.00,7700000.00,7625000.00,540088.14,450000.00,367118.70,4.02,34.59
2,bedrooms,0.00,33.00,33.00,3.37,3.00,0.93,1.97,49.06
3,bathrooms,0.00,8.00,8.00,2.11,2.25,0.77,0.51,1.28
4,sqft_living,290.00,13540.00,13250.00,2079.90,1910.00,918.42,1.47,5.24
5,sqft_lot,520.00,1651359.00,1650839.00,15106.97,7618.00,41419.55,13.06,285.08
6,floors,1.00,3.50,2.50,1.49,1.50,0.54,0.62,-0.48
7,waterfront,0.00,1.00,1.00,0.01,0.00,0.09,11.39,127.63
8,view,0.00,4.00,4.00,0.23,0.00,0.77,3.40,10.89
9,condition,1.00,5.00,4.00,3.41,3.00,0.65,1.03,0.53


### 1.4.2 Variáveis Categóricas

In [10]:
cat_atributos.columns 

Index([], dtype='object')

Não temos variáveis categóricos neste dataset

# 2.0 Feature Engineering

In [11]:
df2 = df.copy()

## 2.1 Mapa de Hípoteses

In [12]:
# Vou fazer essa parte
#Image('../img/')

## 2.2 Criação de Hípoteses

## 2.1 Mapa de Hípoteses

## 2.3 Feature Engineering

### 2.3.1 Condition house

In [13]:
#Condition house
df2['condition_type']= 'NA'
df2['condition_type']= df2['condition'].apply(lambda x: 'bad'  if x <= 2 else 
                                            'regular' if (x >= 3) & (x <= 4) 
                                            else 'good')

### 2.3.2 Level

In [14]:
#Level
df2['level']= 'NA'
df2['level']= df2['price'].apply(lambda x: 0  if x < 321950 else 
                                              1  if (x >= 321950) & (x < 450000) else 
                                              2  if (x >= 450000) & (x < 645000) else 3)  
df2['level'] = df2['level'].astype(int)

### 2.3.3 Size

In [15]:
#Size
df2['size']= 'NA'
df2['size']= df2['sqft_living'].apply(lambda x: 0  if x < 1427 else 
                                              1  if (x >= 1427) & (x < 1910) else 
                                              2  if (x >= 1910) & (x < 2550) else 3)  
df2['size'] = df2['size'].astype(int)

### 2.3.4 Waterfront

In [16]:
#Waterfront
df2['is_waterfront'] = df2['waterfront'].apply( lambda x: 'yes' if x == 1 else 'no' )

### 2.3.5 Inserir localização no Dataset

In [17]:
import time
from multiprocessing import Pool 

In [22]:
df2['query']= df2[['lat','long']].apply(lambda x: str(x['lat']) + ',' + str(x['long']),axis=1)
df3 = df2[['id', 'query']]

In [ ]:
import defs

p = Pool(2)

for i in range( len( df3 ) ):
    print( 'Loop: {}/{}'.format( i, len( df3 ) ) )
    start = time.process_time()
    df3[['place_id','osm_type','country','country_code']] = p.map(defs.get_latlong,df3.iterrows())

end = time.process_time()
print('Time Elapsed: {}',end - start)

Loop: 0/21613


In [21]:
df3.head()

,id,query,place_id,osm_type,country,country_code
0,7129300520,"47.5112,-122.257",148421265,way,United States,us
1,6414100192,"47.721,-122.319",148033904,way,United States,us
2,5631500400,"47.7379,-122.233",76736995,node,United States,us
3,2487200875,"47.5208,-122.393",145042219,way,United States,us
4,1954400510,"47.6168,-122.045",292831047,way,United States,us


# 3.0 Analise Expliratória